# Tests of algorithms.tensorflow.weight_manager.py

In [1]:
from typing import List
import tensorflow as tf
import numpy as np
from tensorflow.keras import datasets, layers, models

import weight_manager

In [2]:
# simple model creation

model = tf.keras.Sequential()
model.add(layers.Dense(units=10, input_shape=(1,)))
model.add(layers.Dense(units=10))
# model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32,32,3)))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))


In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                20        
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
Total params: 130
Trainable params: 130
Non-trainable params: 0
_________________________________________________________________


## Analysis of model.weights

In [4]:
model.weights 
# list of tf.Variable, 1 for each layer, therefore we use this python object

# model.weights returns :
# [layer0_kernel,layer0_bias, ..., layerN_kernel,layerN_bias]
# so parameters are the kernels and biases of each layer 

[<tf.Variable 'dense/kernel:0' shape=(1, 10) dtype=float32, numpy=
 array([[ 0.6143891 , -0.33047158, -0.24180108,  0.5252829 ,  0.04985142,
         -0.5246601 , -0.70460457,  0.12846226,  0.48184675,  0.2796244 ]],
       dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>,
 <tf.Variable 'dense_1/kernel:0' shape=(10, 10) dtype=float32, numpy=
 array([[-0.02694577, -0.3598944 ,  0.18268394,  0.3832196 ,  0.39344966,
         -0.3509472 ,  0.0593583 , -0.07464445, -0.2811039 ,  0.30090827],
        [-0.35939425, -0.19665053, -0.43978634,  0.19091892, -0.23696375,
          0.51201093,  0.5377995 ,  0.27957994,  0.34323692,  0.12575239],
        [ 0.3113979 , -0.0096761 , -0.29562196,  0.31374955, -0.08146018,
          0.07152629,  0.02608532, -0.3196398 , -0.12092614,  0.16682756],
        [-0.2533424 ,  0.33970338, -0.47341517,  0.44609904,  0.04294217,
          0.19805825, -0.06974143,  0.113

In [5]:
var = tf.Variable(initial_value=np.full(shape=(10,), fill_value=1.), dtype='float32')

# it is important to force dtype float32 of weights updates

In [6]:
var.value()

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)>

Native methods for weights update

In [7]:
model.weights[1].assign_add(delta = var) # sums weight and delta

<tf.Variable 'UnreadVariable' shape=(10,) dtype=float32, numpy=array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)>

In [8]:
model.weights[1].assign_sub(delta = var) # sums weight and delta

<tf.Variable 'UnreadVariable' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>

## Tests of weight_manager module

In [9]:
updates = [
    tf.Variable(initial_value=np.full(shape=(1,10), fill_value=1., dtype='float32')),
    tf.Variable(initial_value=np.full(shape=(10,), fill_value=1., dtype='float32')),
    tf.Variable(initial_value=np.full(shape=(10,10), fill_value=1., dtype='float32')),
    tf.Variable(initial_value=np.full(shape=(10,), fill_value=1., dtype='float32')),
]

# dtype='float32' is important

In [10]:
updates2 = [
    tf.Variable(initial_value=np.full(shape=(1,10), fill_value=2., dtype='float32')),
    tf.Variable(initial_value=np.full(shape=(10,), fill_value=2., dtype='float32')),
    tf.Variable(initial_value=np.full(shape=(10,10), fill_value=2., dtype='float32')),
    tf.Variable(initial_value=np.full(shape=(10,), fill_value=2., dtype='float32')),
]

In [11]:
weight_manager.increment_parameters(model=model, updates=updates, updates_multiplier=0.5)

In [12]:
model.weights # on bias layers we see clearly the results

[<tf.Variable 'dense/kernel:0' shape=(1, 10) dtype=float32, numpy=
 array([[ 1.1143892 ,  0.16952842,  0.25819892,  1.0252829 ,  0.5498514 ,
         -0.02466011, -0.20460457,  0.62846226,  0.98184675,  0.7796244 ]],
       dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(10,) dtype=float32, numpy=array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5], dtype=float32)>,
 <tf.Variable 'dense_1/kernel:0' shape=(10, 10) dtype=float32, numpy=
 array([[ 0.47305423,  0.1401056 ,  0.68268394,  0.8832196 ,  0.89344966,
          0.1490528 ,  0.5593583 ,  0.42535555,  0.21889609,  0.80090827],
        [ 0.14060575,  0.30334947,  0.06021366,  0.6909189 ,  0.26303625,
          1.0120109 ,  1.0377995 ,  0.77957994,  0.8432369 ,  0.6257524 ],
        [ 0.8113979 ,  0.4903239 ,  0.20437804,  0.81374955,  0.41853982,
          0.5715263 ,  0.5260853 ,  0.1803602 ,  0.37907386,  0.66682756],
        [ 0.24665761,  0.8397034 ,  0.02658483,  0.94609904,  0.54294217,
          0.69805825,  0.430258

In [13]:
weight_manager.weighted_sum_parameters([updates, updates2], [0.5, 0.2])
# incompréhensible

[<tf.Tensor: shape=(1, 10), dtype=float32, numpy=array([[0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9]], dtype=float32)>,
 <tf.Tensor: shape=(10,), dtype=float32, numpy=array([0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9], dtype=float32)>,
 <tf.Tensor: shape=(10, 10), dtype=float32, numpy=
 array([[0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9],
        [0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9],
        [0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9],
        [0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9],
        [0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9],
        [0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9],
        [0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9],
        [0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9],
        [0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9],
        [0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9]], dtype=float32)>,
 <tf.Tensor: shape=(10,), dtype=float32, numpy=array([0.9, 0.9, 0.9, 0.9, 0.9, 0.9,

In [14]:
.5 * 1. + .2 * 2.

0.9

In [15]:
weight_manager.subtract_parameters(updates2, updates)

[<tf.Tensor: shape=(1, 10), dtype=float32, numpy=array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>,
 <tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)>,
 <tf.Tensor: shape=(10, 10), dtype=float32, numpy=
 array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>,
 <tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)>]

In [16]:
weight_manager.add_parameters(updates2, updates)

[<tf.Tensor: shape=(1, 10), dtype=float32, numpy=array([[3., 3., 3., 3., 3., 3., 3., 3., 3., 3.]], dtype=float32)>,
 <tf.Tensor: shape=(10,), dtype=float32, numpy=array([3., 3., 3., 3., 3., 3., 3., 3., 3., 3.], dtype=float32)>,
 <tf.Tensor: shape=(10, 10), dtype=float32, numpy=
 array([[3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
        [3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
        [3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
        [3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
        [3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
        [3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
        [3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
        [3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
        [3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
        [3., 3., 3., 3., 3., 3., 3., 3., 3., 3.]], dtype=float32)>,
 <tf.Tensor: shape=(10,), dtype=float32, numpy=array([3., 3., 3., 3., 3., 3., 3., 3., 3., 3.], dtype=float32)>]

In [17]:
old_weights = model.get_weights() # copy

model.set_weights(updates)

model.weights

[<tf.Variable 'dense/kernel:0' shape=(1, 10) dtype=float32, numpy=array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(10,) dtype=float32, numpy=array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)>,
 <tf.Variable 'dense_1/kernel:0' shape=(10, 10) dtype=float32, numpy=
 array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)>]

In [18]:
model.set_weights(old_weights)

model.weights

[<tf.Variable 'dense/kernel:0' shape=(1, 10) dtype=float32, numpy=
 array([[ 1.1143892 ,  0.16952842,  0.25819892,  1.0252829 ,  0.5498514 ,
         -0.02466011, -0.20460457,  0.62846226,  0.98184675,  0.7796244 ]],
       dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(10,) dtype=float32, numpy=array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5], dtype=float32)>,
 <tf.Variable 'dense_1/kernel:0' shape=(10, 10) dtype=float32, numpy=
 array([[ 0.47305423,  0.1401056 ,  0.68268394,  0.8832196 ,  0.89344966,
          0.1490528 ,  0.5593583 ,  0.42535555,  0.21889609,  0.80090827],
        [ 0.14060575,  0.30334947,  0.06021366,  0.6909189 ,  0.26303625,
          1.0120109 ,  1.0377995 ,  0.77957994,  0.8432369 ,  0.6257524 ],
        [ 0.8113979 ,  0.4903239 ,  0.20437804,  0.81374955,  0.41853982,
          0.5715263 ,  0.5260853 ,  0.1803602 ,  0.37907386,  0.66682756],
        [ 0.24665761,  0.8397034 ,  0.02658483,  0.94609904,  0.54294217,
          0.69805825,  0.430258

In [20]:
weight_manager.zeros_like_parameters(model)

[<tf.Tensor: shape=(1, 10), dtype=float32, numpy=array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>,
 <tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>,
 <tf.Tensor: shape=(10, 10), dtype=float32, numpy=
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>,
 <tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>]

In [21]:
model.weights

[<tf.Variable 'dense/kernel:0' shape=(1, 10) dtype=float32, numpy=
 array([[ 1.1143892 ,  0.16952842,  0.25819892,  1.0252829 ,  0.5498514 ,
         -0.02466011, -0.20460457,  0.62846226,  0.98184675,  0.7796244 ]],
       dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(10,) dtype=float32, numpy=array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5], dtype=float32)>,
 <tf.Variable 'dense_1/kernel:0' shape=(10, 10) dtype=float32, numpy=
 array([[ 0.47305423,  0.1401056 ,  0.68268394,  0.8832196 ,  0.89344966,
          0.1490528 ,  0.5593583 ,  0.42535555,  0.21889609,  0.80090827],
        [ 0.14060575,  0.30334947,  0.06021366,  0.6909189 ,  0.26303625,
          1.0120109 ,  1.0377995 ,  0.77957994,  0.8432369 ,  0.6257524 ],
        [ 0.8113979 ,  0.4903239 ,  0.20437804,  0.81374955,  0.41853982,
          0.5715263 ,  0.5260853 ,  0.1803602 ,  0.37907386,  0.66682756],
        [ 0.24665761,  0.8397034 ,  0.02658483,  0.94609904,  0.54294217,
          0.69805825,  0.430258

## Conclusion
primitives of weight_manager work with List[tf.Variable] objects 